# Score de pagamentos Itau Cartões A03

Esse notebook tem por objetivo criar um modelo de classificação de probabilidade de pagamentos para o segmento A03 da carteira Itau Cartões da Zanc.
Faremos uso de uma base de cerca de 100 mil cpfs que tiveram permanencia de 45 completos dentro da empresa.
Nosso target é conseguir distinguir quem tem mais chances de recuperação de crédito de quem não tem.

Hoje já obtivemos algum resultado utilizando Logistic Regression através da ferramenta Orange.
A tarefa inicial será de transportar para esse notebookk com código python o que foi feito no Orange.
Na seqüência será proposto uma outra abordagem utilizando Gradient Boosting

## Preparação dos dados

#### Importando módulos necessários

In [68]:
import numpy as np # Sem ele não somos ninguém
import pandas as pd # Para carregar os dados em um datafram
from sklearn.utils import resample # Para balancear os dados
from sklearn.impute import SimpleImputer # Para preencher os campos que estivem com valores nulos
from sklearn.preprocessing import MinMaxScaler # Para normalização dos campos numéricos
from sklearn.linear_model import LogisticRegression # Para instanciar o modelo de Regressão Logística
from sklearn.ensemble import GradientBoostingClassifier # Para instanciar o modelo de Gradient Boosting 
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split # Para avaliação dos modelos

#### Importando os dados para um df pandas

Vamos importar o dataframe utilizando a funcao read csv padrão do pandas, analisando as colunas, tipos e valores nulos.

In [69]:
df = pd.read_csv("data\exportar.csv", encoding="latin1", delimiter=";")

In [57]:
df.head()

,data_status_boletagem,LOJA,LOJA TRATADO,scorecontratante,dataentrada,validadecampanha,atrasocongelado,valorcartacampanha,vlclusters,status_boletagem,desconto,bandeira,publico,matriz,Pagamentos
0,0,ITAUCARD HIPER 2.0,OUTROS,242,22/02/2019 00:00,18/04/2019,204,"207,85","276,47",BOLETAR_A_VONTADE,"48,19",CC,Não definido,3,0
1,0,CARTÃO EXTRA 2.0,CARTÃO EXTRA 2.0,246,15/11/2018 00:00,09/01/2019,304,"1232,75","2612,6",BOLETAR_A_VONTADE,"71,55",FC,Não definido,4,0
2,0,MAGAZINE LUIZA/LUIZACRED FLEX,MAGAZINE LUIZA/LUIZACRED FLEX,248,17/11/2018 00:00,11/01/2019,361,"1106,69","2407,57",BOLETAR_A_VONTADE,"77,65",LC,Não definido,3,0
3,0,MAGAZINE LUIZA/LUIZACRED FLEX,MAGAZINE LUIZA/LUIZACRED FLEX,253,22/04/2019 00:00,14/06/2019,367,"2511,74","9817,66",BOLETAR_A_VONTADE,"86,74",LC,Não definido,3,0
4,0,CARTÃO WALMART 2.0,CARTÃO WALMART 2.0,253,15/02/2019 00:00,11/04/2019,362,"1320,07","2918,04",BOLETAR_A_VONTADE,"77,28",HC,Não definido,3,0


In [7]:
df.shape

(111420, 15)

In [8]:
df.dtypes

data_status_boletagem    object
LOJA                     object
LOJA TRATADO             object
scorecontratante          int64
dataentrada              object
validadecampanha         object
atrasocongelado           int64
valorcartacampanha       object
vlclusters               object
status_boletagem         object
desconto                 object
bandeira                 object
publico                  object
matriz                    int64
Pagamentos                int64
dtype: object

In [9]:
df.isna().sum()

data_status_boletagem    0
LOJA                     0
LOJA TRATADO             0
scorecontratante         0
dataentrada              0
validadecampanha         0
atrasocongelado          0
valorcartacampanha       0
vlclusters               0
status_boletagem         0
desconto                 0
bandeira                 0
publico                  0
matriz                   0
Pagamentos               0
dtype: int64

#### Convertendo campos numéricos

In [70]:
df.iloc[:,7] = df.iloc[:,7].str.replace(',', '.').astype(float)
df.iloc[:,8] = df.iloc[:,8].str.replace(',', '.').astype(float)
df.iloc[:,10] = df.iloc[:,10].str.replace(',', '.').astype(float)

#### Retirando valores > 2,5mil, pois serão cobrados na célula de AV

In [71]:
df = df[df['valorcartacampanha'] < 2500]

#### Normalizando campos numéricos de valores de dívida

In [72]:
escalador = MinMaxScaler()
x = df[['valorcartacampanha']].values
escalador.fit(x)
x_transformado = escalador.transform(x)
df[['valorcartacampanha']] = x_transformado

In [73]:
x = df[['vlclusters']].values
escalador.fit(x)
x_transformado = escalador.transform(x)
df[['vlclusters']] = x_transformado

In [74]:
x = df[['desconto']].values
escalador.fit(x)
x_transformado = escalador.transform(x)
df[['desconto']] = x_transformado

#### Dropando colunas de data e coluna já mapeada (Loja)

In [63]:
df.pop('data_status_boletagem')
df.pop('LOJA')
df.pop('validadecampanha')
df.pop('dataentrada')

0         22/02/2019 00:00
1         15/11/2018 00:00
2         17/11/2018 00:00
4         15/02/2019 00:00
5         30/11/2018 00:00
6         09/03/2019 00:00
7         02/03/2019 00:00
8         21/02/2019 00:00
10        14/02/2019 00:00
11        22/02/2019 00:00
13        22/03/2019 00:00
15        21/02/2019 00:00
17        05/04/2019 00:00
18        14/03/2019 00:00
19        16/03/2019 00:00
20        24/01/2019 00:00
21        21/03/2019 00:00
22        06/02/2019 00:00
23        10/01/2019 00:00
24        29/12/2018 00:00
25        23/11/2018 00:00
26        27/02/2019 00:00
27        22/02/2019 00:00
28        14/03/2019 00:00
29        05/01/2019 00:00
30        24/01/2019 00:00
31        23/11/2018 00:00
32        18/01/2019 00:00
33        10/01/2019 00:00
34        30/11/2018 00:00
                ...       
111377    07/12/2018 00:00
111379    07/12/2018 00:00
111380    07/12/2018 00:00
111381    07/12/2018 00:00
111382    07/12/2018 00:00
111384    07/12/2018 00:00
1

#### Ajustando valores da coluna público

In [64]:
df['publico'].value_counts()

Não definido           56807
elegivel_excecao       19476
Elegível Exceção        9021
Elegivel Excecao        2249
alto_atrito             1166
Alto Atrito              586
Eleg?¡vel Exce?º?úo      121
Name: publico, dtype: int64

In [65]:
def converte_publico(x):
    if x=="Eleg?¡vel Exce?º?úo" or x=="Elegível Exceção" or x=="Elegivel Excecao":
        return "elegivel_excecao"
    elif x=="Alto Atrito":
        return "alto_atrito"
    else:
        return x

df['publico'] = df['publico'].apply(converte_publico)
df['publico'].value_counts()

Não definido        56807
elegivel_excecao    30867
alto_atrito          1752
Name: publico, dtype: int64

#### Transformando colunas categóricas em binárias

In [66]:
df_dummies = pd.get_dummies(df_balanceado)
df_dummies.columns

Index(['scorecontratante', 'atrasocongelado', 'valorcartacampanha',
       'vlclusters', 'desconto', 'matriz',
       'LOJA TRATADO_CARTAO PL EMBANDEIRADO MARISA',
       'LOJA TRATADO_CARTAO PL FIC CB S/P',
       'LOJA TRATADO_CARTAO PL FIC EXTRA BAND',
       'LOJA TRATADO_CARTAO PL FIC EXTRA S/P', 'LOJA TRATADO_CARTÃO EXTRA 2.0',
       'LOJA TRATADO_CARTÃO MARISA 2.0', 'LOJA TRATADO_CARTÃO PONTO FRIO 2.0',
       'LOJA TRATADO_CARTÃO PRIV LBL FIC ASSAI',
       'LOJA TRATADO_CARTÃO WALMART 2.0', 'LOJA TRATADO_CREDICARD CLASSICOS',
       'LOJA TRATADO_HIPERCARD', 'LOJA TRATADO_ITAUCARD 2.0 CANAIS DIRETOS',
       'LOJA TRATADO_MAGAZINE LUIZA/LUIZACRED FLEX',
       'LOJA TRATADO_OPERACOES CREDITO CREDICARD', 'LOJA TRATADO_OUTROS',
       'LOJA TRATADO_TAM ITAUCARD 2.0', 'status_boletagem_BOLETAR_A_PARTIR_',
       'status_boletagem_BOLETAR_A_VONTADE', 'bandeira_CC', 'bandeira_CR',
       'bandeira_FA', 'bandeira_FC', 'bandeira_HC', 'bandeira_LC',
       'bandeira_MA', 'publico_Não

#### Separando treino e teste

In [67]:
df_dummies['Pagamentos'].value_counts()

KeyError: 'Pagamentos'

In [ ]:
df_maioria = df_dummies[df_dummies.Pagamentos==0]
df_minoria = df_dummies[df_dummies.Pagamentos==1]
df_maioria_randomizado = resample(df_maioria, 
                                 replace=True,     
                                 n_samples=2073,    
                                 random_state=123)
df_balanceado = pd.concat([df_maioria_randomizado, df_minoria])
df_balanceado.Pagamentos.value_counts()

In [ ]:
y_train = df_balanceado.pop('Pagamentos').values
X_train = df_balanceado.values
y_test = df_dummies.pop('Pagamentos').values
X_test = df_dummies.values

#### Montando grid dos hiperparâmetros

#### To-do

~~1. Antes de gerar as dummies montar uma transformação para a coluna publico corrigindo:
'publico_Alto Atrito', 'publico_Eleg?¡vel Exce?º?úo', 'publico_Elegivel Excecao', 'publico_Elegível Exceção','publico_Não definido', 'publico_alto_atrito','publico_elegivel_excecao'~~
2. Ajutar a métrica para ROC
3. Rodar GridSearch para os hiperparâmetros da regLog e Xgboost otimizando ROC
4. Selecionar o modelo e montar .py para predizer através do DW
5. Integrar com DW para criar uma coluna com a probabilidade prevista
6. Criar tratamento para nulos, pois eles foram retirados nessa amostra e será necessário para treinar no futuro
